# Versiones Públicas de Sentencias - Divorcios

Antes de ejecutar lea README.md y siga los pasos descritos.

In [ ]:
from dotenv import load_dotenv
from IPython.display import display
import os
import pandas as pd
import requests

In [ ]:
# Rango de fechas a consultar
FECHA_DESDE = "2021-08-16"
FECHA_HASTA = "2021-09-24"

In [ ]:
# Hola a Plataforma Web API OAuth2
HOST = "https://plataforma-web-api-oauth2-dot-pjecz-268521.uc.r.appspot.com"
respuesta = requests.get(HOST)
print(respuesta.status_code)
print(respuesta.text)

In [ ]:
# Cargar username/password
load_dotenv()
parametros = {
    "username": os.getenv("USERNAME"),
    "password": os.getenv("PASSWORD"),
}
print(parametros["username"])

In [ ]:
# Solicitar token
cabeceras_para_autentificar = {'content-type': "application/x-www-form-urlencoded"}
respuesta = requests.post(f"{HOST}/token", data=parametros, headers=cabeceras_para_autentificar)
print(respuesta.status_code)

In [ ]:
# Definir cabeceras_para_api
cabeceras_para_api = None
if respuesta.status_code == 200:
    token = respuesta.json()['access_token']
    cabeceras_para_api = {'Authorization': 'Bearer ' + token}
print(cabeceras_para_api)

In [ ]:
# Función consultar
def consultar(ruta: str, parametros: dict = None):
    """Consultar la API entrega dos valores: listado/registro, total"""
    try:
        respuesta = requests.get(f"{HOST}/v1/{ruta}", headers=cabeceras_para_api, params=parametros, timeout=12)
    except requests.exceptions.RequestException as error:
        raise Exception("Parece que hay un error en la consulta: " + str(error))
    #print(respuesta.status_code)
    if respuesta.status_code != 200:
        raise requests.HTTPError("Parece que ya caducó el token")
    datos = respuesta.json()
    if "items" in datos:
        return pd.json_normalize(datos["items"]), datos["total"]
    return pd.json_normalize(datos), 1

In [ ]:
# Materias
datos, total = consultar("materias")
datos

In [ ]:
# Tipos de Juicios de la Materia FAMILIAR ORAL (6)
datos, total = consultar("materias/6/tipos_juicios")
datos

In [ ]:
# Tipo de Juicio DIVORCIO (216)
datos, total = consultar("materias/6/tipos_juicios/216")
datos

In [ ]:
# Sentencias con Tipo de Juicio DIVORCIO (216)
parametros = {"materia_tipo_juicio_id": 216, "fecha_desde": FECHA_DESDE, "fecha_hasta": FECHA_HASTA}
datos, total = consultar("sentencias", parametros)
total

In [ ]:
# Distritos
datos, total = consultar("distritos")
datos

In [ ]:
# Autoridades del Distrito Judicial
# Para elegir el distrito, descomente la linea que define su valor para los siguientes bloques
# DISTRITO_ID = 1  # Distrito Judicial de Acuña (1)
# DISTRITO_ID = 2  # Distrito Judicial de Monclova (2)
# DISTRITO_ID = 3  # Distrito Judicial de Parras de la Fuente (3)
# DISTRITO_ID = 4  # Distrito Judicial de Río Grande (Piedras Negras) (4)
# DISTRITO_ID = 5  # Distrito Sabinas (5)
# DISTRITO_ID = 6  # Distrito Judicial de Saltillo (6)
# DISTRITO_ID = 7  # Distrito Judicial de San Pedro de las Colonias (7)
DISTRITO_ID = 8  # Distrito Judicial de Torreón (8)
# MATERIA_ID = 6  # Materia FAMILIAR ORAL (6)
parametros = {"distrito_id": DISTRITO_ID}
datos, total = consultar("autoridades", parametros)
datos[["id", "distrito_nombre", "descripcion", "clave", "materia_nombre"]]

In [ ]:
# Sentencias del Distrito Judicial en el rango de fechas
parametros = {"distrito_id": DISTRITO_ID, "fecha_desde": FECHA_DESDE, "fecha_hasta": FECHA_HASTA, "limit": 500}
sentencias, total = consultar("sentencias", parametros)
print("Total", total)
if total == 0:
    raise ValueError("No hay sentencias en el distrito judicial y en el rango de fechas dado")
sentencias.materia_nombre = sentencias.materia_nombre.astype("category")
sentencias.materia_tipo_juicio_descripcion = sentencias.materia_tipo_juicio_descripcion.astype("category")
sentencias[["id", "distrito_nombre_corto", "autoridad_descripcion_corta", "sentencia_fecha", "sentencia", "expediente", "url"]].head(8)

In [ ]:
# Reporte: filtrar por materia
sentencias.materia_nombre.cat.categories
#sentencias_familiar_oral = sentencias[sentencias.materia_nombre == 'FAMILIAR ORAL']
#sentencias_familiar_oral[["id", "distrito_nombre_corto", "autoridad_descripcion_corta", "sentencia_fecha", "sentencia", "expediente", "descripcion", "url"]]

In [ ]:
# Reporte: agrupado por materia
agrupado = sentencias.groupby("materia_nombre")
# agrupado = sentencias.groupby(["materia_nombre", "materia_tipo_juicio_descripcion", "autoridad_descripcion_corta"])
agrupado["autoridad_descripcion_corta"].count()

In [ ]:
# Reporte: pivot table
sentencias.pivot_table(
    index=["materia_tipo_juicio_descripcion"], 
    columns=pd.Grouper(key="autoridad_descripcion_corta"), 
    values="autoridad_clave", 
    aggfunc='count',
)

In [ ]:
# Reporte: bucle de pivot table
for materia in sentencias.materia_nombre.cat.categories:
    sentencias_por_materia = sentencias[sentencias.materia_nombre == materia]
    print(materia, len(sentencias_por_materia))
    display(
        sentencias_por_materia.pivot_table(
            index=["materia_tipo_juicio_descripcion"], 
            columns=pd.Grouper(key="autoridad_descripcion_corta"), 
            values="autoridad_clave", 
            aggfunc='count',
        )
    )